In [ ]:
'''
Analyze force-xxx experiement of ATI Nano-17 force sensor 
Author: Yitian Shao
Created on 2022.09.29 based on 'NanoForceSensor'
'''

# %matplotlib notebook
# %matplotlib notebook 

import time
from os import walk
import os.path as ospa
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib import mlab
import pandas as pd
import scipy.io as scio
from scipy import signal
import seaborn as sns

plt.rc('font', size=10, family='Verdana') # 'Tahoma', 'DejaVu Sans', 'Verdana'"
plt.rc('axes', edgecolor='k', linewidth=0.75, labelcolor='k')
plt.rc('axes.spines', **{'bottom':True, 'left':True, 'right':True, 'top':True})
plt.rcParams['xtick.top'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['errorbar.capsize'] = 4

figSize_inch = (3.2, 2.4)

''' Define Color Here '''
pltBlue = (32/255,120/255,180/255)
pltGreen = (32/255,180/255,120/255)
pltRed = (220/255,95/255,87/255)
pltYellow = (200/255,180/255,50/255)
pltPurple = (100/255,50/255,180/255)

'''  Color used in the paper '''
pBlue = (32/255,120/255,180/255)
pRed = (185/255,49/255,49/255)
pGrey = (77/255,77/255,77/255)

''' Remove warnings '''
import warnings
warnings.filterwarnings('ignore')

In [ ]:
'''
General Functions
'''

def aPlot(figName='', is3D = False, dpi=72):
    ax = []
    
    fig1 = plt.figure(figsize = (6,3), dpi=dpi)
    
    fig1.suptitle(figName, fontsize=16)
    if(is3D):
        ax = fig1.add_subplot(111, projection='3d')
    else:
        ax = fig1.add_subplot(111)
        
    return ax, fig1

def lowpassSmooth(datain, cutFreqRatio = 0.05, order = 8):
    b, a = signal.butter(order, 2 * cutFreqRatio, btype='low')
    dataout = signal.filtfilt(b, a, datain)
    return dataout

def movAvgSmooth(datain, winLen=100):
    dataout = np.convolve(datain, np.ones(winLen)/winLen, mode='same')
    return dataout

def decodeData(fileName, numFormat, frontCode='', rearCode='', isString=False):
    segStr = re.findall(frontCode+numFormat+rearCode, fileName)
    if segStr:
        if isString:
            return segStr[0]
        else:
            numData = float(re.findall(numFormat, segStr[0])[0])
    else:
        numData = None
    return numData

def cutRepeatTrial(datain, Fs, expectedTrialNum, disp=False, axis=2, discardInd=500, cutFreq=10):
    rawData = -datain[:,axis]
    
    rawData[rawData < -0.1] = 0; # Remove negative value
    rawData = rawData[discardInd:] # Discard the beginning
    
    smData = lowpassSmooth(rawData, cutFreqRatio = (cutFreq/Fs), order = 8)
    
    smData = signal.detrend(smData, type='linear')
    smData = smData - smData[0]
    
    samp = np.arange(len(rawData))
    
    maxRawValue = np.sqrt(np.mean(smData**2))
    
    ''' Fine tunning needed for individual measurement session'''
    segPointInd = np.squeeze(np.argwhere(smData > 2 * maxRawValue)) # Find value larger than ?% of peak as valid segment data point
    segGapInd = np.squeeze(np.argwhere(np.diff(segPointInd) > 1))# Index of point where large gap occurs (end and start of a seg)
    cutInd = (0.5 * (segPointInd[segGapInd] + segPointInd[segGapInd+1])).astype(int) # Cut in the middle of a end and a start point
    
    if not isinstance(cutInd,np.ndarray) or len(cutInd) < expectedTrialNum-1:
        if disp:
            ax0, _ = aPlot(); 
            ax0.plot(samp, rawData, color='k'); 
            plt.show();       
        return []
    
    if len(cutInd) > expectedTrialNum-1:
        cutInd = cutInd[:expectedTrialNum]
    elif len(cutInd) == expectedTrialNum-1:
        cutInd = np.insert(cutInd, expectedTrialNum-1, len(rawData)-1)

    avgSegLen = np.mean(np.diff(cutInd))
    cutInd = np.insert(cutInd, 0, max(cutInd[0]-avgSegLen, 0))
    
    if disp:
        ax0, _ = aPlot(); 
        ax0.plot(samp, rawData, color='k'); 
        ax0.plot(samp, smData, color='tab:blue')
        ax0.plot(cutInd, np.zeros(cutInd.shape), '*r')
        plt.show();
    
    return cutInd

def getDataArray(dFrame, indList, colName, toFlat=False):
    compValue = (dFrame.loc[indList,colName]).to_numpy()  
    
    if toFlat:
        flatValue = []
        for aRow in compValue:
            flatValue.extend(aRow)
        flatValue = np.array(flatValue)
        return flatValue
            
    return compValue
    
def getMeanSTD(compValue): # Compute errorbar of data mixing all conditions (rows of input array)
    if len(compValue) == 0:
        return None, None
        
    compValue = np.array(compValue)
    
    meanValue = np.mean(compValue)
    stdValue = np.std(compValue)
    
    return meanValue, stdValue

def segmentByEnergy(datain, segIntervalSamp, disp=False, threshold=0.1):
    ''' This function segment the accelerometer data based on detecting the energy surge of the input signal '''
    # 'datain' - a data vector used to determine the segmentation
    # 'segIntervalSamp' - the number of samples for each detection window (sliding window length)
    # 'disp' - display the segmentation result if True
    # 'threshold' - the threshold to determine whether the energy has increased or decreased significantly
    
    dataEnergy = datain
    
    smoothSig = movAvgSmooth(dataEnergy, segIntervalSamp) # Identify the energy fluctuation with moving-average smoothing
        
    samp = np.arange(len(datain)-1) # Shorten by 1 samples for indexing after computing the 1st difference
    energyDiff = np.diff(smoothSig)
    energyIncreaseInd = samp[energyDiff > threshold] # Index of samples when the energy increased significantly
    energyDecreaseInd = samp[energyDiff < -threshold] # Index of samples when the energy decreased significantly
    
    tmp = energyIncreaseInd[:-1]
    energySurgeUpInd = tmp[np.diff(energyIncreaseInd) > (segIntervalSamp * 0.5)] # Enforce a minimal duration bewteen two consecutive energy increasing intervals
    
    tmp = energyDecreaseInd[:-1]
    energySurgeDownInd = tmp[np.diff(energyDecreaseInd) > (segIntervalSamp * 0.5)] # Enforce a minimal duration bewteen two consecutive energy decreasing intervals

    if disp:
#         _,ax0 = plt.subplots(1,1,figsize=(14, 6),dpi=72); 
        ax0, _ = aPlot(); 
        ax0.plot(samp, datain[:-1], color='tab:grey'); 
        axb = ax0.twinx() 
#         axb.plot(samp, smoothSig[:-1], color='tab:green') # Plot the smoothed signals
#         axb.tick_params(axis='y', labelcolor='tab:green')
        ax0.plot(energySurgeUpInd, np.zeros(energySurgeUpInd.shape), '.r')
        ax0.plot(energySurgeDownInd, np.zeros(energySurgeDownInd.shape), '.b')
        plt.show();
        
    return energySurgeUpInd, energySurgeDownInd


'''
Load, preprocess and segment force sensor data
'''
def dataSegmentation(measureDataPath):
    nanoData = []
    ''' Data preprocess and segmentation '''
    for root, directories, files in walk(measureDataPath):
        for fileName in files:
            measureDate = decodeData(fileName, '\d{4}', rearCode='2022')
            tubeLen = decodeData(fileName, '[\d+\.]*\d+', rearCode='mm')
            infillVol = decodeData(fileName, '[\d+\.]*\d+', rearCode='mL') 
            pulseFreq = decodeData(fileName, '[\d+\.]*\d+', rearCode='mHz')  
            
            voltage = decodeData(fileName, '[\d+\.]*\d+', rearCode='kV') 
            
            bendAngle = decodeData(fileName, '[\d+\.]*\d+', frontCode='bent_neg')
            if bendAngle is None:
                bendAngle = decodeData(fileName, '[\d+\.]*\d+', frontCode='bent')     
            else:
                bendAngle = -bendAngle 
                
            driveSignal = decodeData(fileName, '*\w+', frontCode='kV_', rearCode='_', isString=True)
            if driveSignal is not None and len(driveSignal)>4:
                driveSignal = driveSignal[3:-1]
            
            trialNum = decodeData(fileName, '\d+', rearCode='.mat') # ForceData20220929
            if trialNum is None:
                trialNum = 1

            isSilInsulated = re.findall('Sil', fileName)
            data = []

            dLabel = ""
            if tubeLen is not None:
                dLabel = dLabel + ("%.1fmm" % tubeLen)    
            if infillVol is not None:
                dLabel = dLabel + ("%.2fmL" % infillVol)           
            if voltage is not None:
                dLabel = dLabel + ("%.1fkV" % voltage)
            if bendAngle is not None:
                dLabel = dLabel + ("%.0fdeg" % bendAngle)
            if driveSignal is not None:
                dLabel = dLabel + driveSignal

            print("%s - trial%s [Fs = ? Hz] %s" % (dLabel, trialNum, driveSignal))

            matData = scio.loadmat(ospa.join(root, fileName))
            if matData:
                forceData = matData['forceNano']

                if re.findall('Frequency', measureDataPath):
                    data.append(forceData)  
                else:
                    Fs = 1000
                    
                    discardInd = 500
                    cutFreq = 10
                    if dLabel == '110.0mm3.0kV60deg':
                        discardInd = 3000
                        cutFreq = 5
                    elif dLabel == '110.0mm7.0kV-20deg':
                        discardInd = 5000
                        cutFreq = 5
                        
                        
                    if driveSignal == 'triangle' or driveSignal == 'ramp':
                        upInd, cutInd = segmentByEnergy(-forceData[:,2], 2500, disp=True, threshold=0.0001)
                    else:
                        cutInd = cutRepeatTrial(forceData, Fs, 5, disp=True, axis=2, discardInd=discardInd, cutFreq=cutFreq)
                    segNum = len(cutInd)-1

                    if segNum > 0: 
                        for j in range(segNum):
                            segData = forceData[cutInd[j]:cutInd[j+1],:]
                            data.append(segData)  

            nanoData.append([dLabel, tubeLen, voltage, bendAngle, trialNum, data, infillVol, driveSignal])

    nanoData = pd.DataFrame(nanoData, columns = ['Label', 'TubeLength_mm', 'Voltage_kV', 'BendAngle_deg', 'Trial','Data', 'Infill_mL', 'DriveSignal'])  

    display(nanoData.head(10))
    display(nanoData.tail(10))
    
    return nanoData

In [ ]:
'''
Pulse train measurements:

Feature extraction from signals: peak value, time-averaged value, rising time
'''

# measureDataPath = './BendAngleData20240419' # Fs = 1000, bending angles [-45,-20,0,20,35,45,60]
measureDataPath = './RampTestData20240418' # driving signal types (pulse, triangle, ramp)

nanoData = dataSegmentation(measureDataPath)

labels = nanoData.Label.unique()
print(labels)

dataFeature = []

for aLabel in labels:
    selectedData = nanoData.loc[(nanoData['Label'] == aLabel)]

    Fs = 1000

    peakF = []
    rawPeakF = []
    steadyRawF = []
    triseF = []
    triseSteady = []
    smF = []
        
    for index, row in selectedData.iterrows():
        for aSignal in row["Data"]:
            smData = lowpassSmooth(-aSignal[:,2], cutFreqRatio = (20/Fs), order = 10) # Use the z-axis force measurement 
            smData = smData - smData[0]

            maxValue = np.max(smData)
            onsetInd = np.argwhere(smData[50:] > 0.10 * maxValue)[0][0]+50 # When signal reach 10% of its peak (onset)
            PTopInd = np.argwhere(smData[50:] > 0.90 * maxValue)[0][0]+50 # When signal reach 90% of its peak
            
            ''' To Find the steady state force '''
            offInd = np.argwhere(smData[50:] > 0.20 * maxValue)[-1][0]+50
            if(offInd == None or offInd < PTopInd):
                offInd = smData.shape[0]
                
            leftInd = PTopInd
            rightInd = offInd
            while(leftInd < rightInd and abs(smData[leftInd] - smData[rightInd]) > (0.05*maxValue)):
                leftInd = leftInd+1
                rightInd = rightInd-1
                
            ''' Over damped case '''
            if (leftInd-PTopInd) > (Fs*0.1):
                print("Over-damped")
                leftInd = PTopInd
                rightInd = offInd

            peakInd = np.argmax(smData)
            peakValue = np.max(smData)           
            rawPeakValue = np.max(-aSignal[:peakInd,2])

            riseTime = (PTopInd - onsetInd)/Fs

            ''' Steady State '''
#             print("Len = %d, leftInd = %d, rightInd = %d" % (len(aSignal[:,2]), leftInd, rightInd))
            steadyRawValue = np.mean(-aSignal[leftInd:rightInd,2]) # steady takes mean during steady state
            steadyRawF.append(steadyRawValue)
            
            peakF.append(peakValue)
            rawPeakF.append(rawPeakValue)

            triseF.append(riseTime)
            
            smF.append(smData)

            onSetSearch = np.argwhere(smData[50:] > 0.10 * 1.0)
            if(len(onSetSearch) < 1):
                print("Cannot determine the rise time")
                riseTimeSteady = 0
            else:
                ind0 = onSetSearch[0][0]+50 # When signal reach 10% of its peak (onset)
                tmp = np.argwhere(smData[50:] > 1.0) # When signal reach 90% of its peak
                ind1 = ind0
                if(len(tmp) > 0):
                    ind1 = tmp[0][0]+50 

                riseTimeSteady = (ind1 - ind0)/Fs
                
            triseSteady.append(riseTimeSteady)
            
            ax1,_ = aPlot(dpi=72)
            ax1.plot(-aSignal[:,2]+aSignal[0,2], 'k')
            ax1.plot([1, peakInd], [rawPeakValue, rawPeakValue], '--c')
            ax1.plot([onsetInd, onsetInd], [0, maxValue], '-r')
            ax1.plot([PTopInd, PTopInd], [0, maxValue], '-r')
        
            ax1.plot([leftInd, leftInd], [0, maxValue], '-c')
            ax1.plot([rightInd, rightInd], [0, maxValue], '-b')
            ax1.plot([leftInd, rightInd], [0, 0], '-b')
            ax1.plot([leftInd, rightInd], [maxValue, maxValue], '-b')
            
            ax1.set_xlabel("%s: peak=%.2fN , steady=%.2fN , riseT=%.3fs" % (aLabel, rawPeakValue,steadyRawValue,riseTime))
            ax1.set_xlim([onsetInd-(0.1*Fs), peakInd + (0.4*Fs)])
            ax1.set_ylabel("Force (N)")
            plt.show()
            
    dataFeature.append([aLabel, row["TubeLength_mm"], row["Voltage_kV"], row["BendAngle_deg"], peakF, rawPeakF, steadyRawF, 
                        triseF, triseSteady, smF, row["DriveSignal"]])

dataFeature = pd.DataFrame(dataFeature, columns = ['Label', 'TubeLength_mm','Voltage_kV','BendAngle_deg','PeakForce',
                                                   'RawPeakForce','steadyRawForce','TriseForce','TriseSteady','Data','DriveSignal'])  

dataFeature.head(10)

In [ ]:
'''
Functions for Computing and Visualizing Errorbar
'''
def computeErrorbar1Var(dframe, xColName, xColList, zColName):
    dMean = []
    dSTD = []
    
    for x in xColList:
        selectInd = (dframe[xColName] == x)

        compValue = getDataArray(dframe, selectInd, zColName, toFlat=True)  

        valueMean, valueSTD = getMeanSTD(compValue)

        dMean.append(valueMean)
        dSTD.append(valueSTD)
    
    dMean = np.array(dMean)
    dSTD = np.array(dSTD)
    
    return dMean, dSTD

def computeErrorbar(dframe, xColName, xColList, yColName, yColList, zColName):
    dMean = []
    dSTD = []
    
    for x in xColList:
        dMean.append([])
        dSTD.append([])
        for y in yColList:
            selectInd = (dframe[xColName] == x) & (dframe[yColName] == y)

            compValue = getDataArray(dframe, selectInd, zColName, toFlat=True)  
            
            valueMean, valueSTD = getMeanSTD(compValue)
            
            dMean[-1].append(valueMean)
            dSTD[-1].append(valueSTD)
    
    dMean = np.array(dMean)
    dSTD = np.array(dSTD)
    
    return dMean, dSTD

def dispErrorbar(ax, x, dMean, dErr, xLabel, xLabelStr, yLabelStr, zLabel, colorPalette=None, style='.-', 
                 dispLegend=True, xShift=[0.1,0.4]):
    yLen = len(yLabelStr)
    
    colors = sns.color_palette(colorPalette, n_colors=yLen)
    
    ax.set_ylabel(zLabel, color='k')
    ax.tick_params(axis='y', labelcolor='k')
    ax.spines['bottom'].set_color('k')
    ax.spines['top'].set_color('k') 
    ax.spines['right'].set_color('k')
    ax.spines['left'].set_color('k')

    pltHandles = []
    for i in np.arange(yLen):
        pltHandles.append(ax.plot(2*x + (i * xShift[0]), dMean[i], style, c=colors[i])[0])
        (ax.errorbar(2*x + (i * xShift[0]), dMean[i], yerr=dErr[i], fmt='none', ecolor=colors[i], capsize=0)[0]) 
        
    ax.set_xlabel(xLabel)
    ax.set_xticklabels(xLabelStr, rotation=0);
    ax.set_xticks(2*x + xShift[1]);

    if dispLegend:
        ax.legend(handles=reversed(pltHandles), labels=reversed(yLabelStr), frameon=False, bbox_to_anchor=(1.02, 1.0), 
                  loc='upper left');
    
    return ax

In [ ]:
'''
(Exclusive) Analysis for Visualizing the Errorbar of multiple conditions (bending angles)
'''

vLevels = [3, 4, 5, 6, 7]

bAngles = [-45, -20, 0, 20, 35, 45, 60]

colorSet = 'hls'
style = '.-'

xLen = len(vLevels)
x = np.arange(xLen)
xLabelStr = np.array(vLevels)
yLabelStr = bAngles
xLabel = 'Voltage (kV)'

meanRawPeakF, stdRawPeakF = computeErrorbar(dataFeature, 'BendAngle_deg', bAngles, 'Voltage_kV', vLevels, 'RawPeakForce')
meanSteadyRawF, stdSteadyRawF = computeErrorbar(dataFeature, 'BendAngle_deg', bAngles, 'Voltage_kV', vLevels, 'steadyRawForce')   

print("Raw Peak force - Mean (n=12, deg)")
print("-45deg: %s \n-20deg: %s \n0deg: %s \n20deg: %s \n35deg: %s \n45deg: %s \n60deg: %s \n" % 
      (meanRawPeakF[0],meanRawPeakF[1],meanRawPeakF[2],meanRawPeakF[3],meanRawPeakF[4],meanRawPeakF[5],meanRawPeakF[6]))
print("Raw Peak force - STD (n=12, deg)")
print("-45deg: %s \n-20deg: %s \n0deg: %s \n20deg: %s \n35deg: %s \n45deg: %s \n60deg: %s \n" % 
      (stdRawPeakF[0],stdRawPeakF[1],stdRawPeakF[2],stdRawPeakF[3],stdRawPeakF[4],meanRawPeakF[5],meanRawPeakF[6]))

meanRawPeakF[meanRawPeakF==None] = 0;
stdRawPeakF[stdRawPeakF==None] = 0;
meanSteadyRawF[meanSteadyRawF==None] = 0;
stdSteadyRawF[stdSteadyRawF==None] = 0;

fig1, ax1 = plt.subplots(dpi=300, figsize=(1,2))
dispErrorbar(ax1, x, meanRawPeakF, stdRawPeakF, xLabel, xLabelStr, yLabelStr, 'Peak Force (N)', colorSet, style=style,
            xShift=[0,0]) # xShift=[0.1,0.05] # Removed xShift on 2024.08.24

fig2, ax2 = plt.subplots(dpi=300, figsize=(1,2))
dispErrorbar(ax2, x, meanSteadyRawF, stdSteadyRawF, xLabel, xLabelStr, yLabelStr, 'Steady State Force (N)', colorSet, style=style,
            xShift=[0,0]) # xShift=[0.1,0.05] # Removed xShift on 2024.08.24

fig1.savefig("force_bendAngle_110.pdf", bbox_inches='tight')
fig2.savefig("force_bendAngle_110_steadystate.pdf", bbox_inches='tight')

In [ ]:
'''
Analysis for Visualizing the Errorbar of multiple conditions (driving signals) - peak and steady state force
'''
driveSignals = ['pulse','triangle','ramp']

colorSet = 'hls'
style = '.-'

xLen = len(driveSignals)
x = np.arange(xLen)
xLabelStr = driveSignals
yLabelStr = 'Force (N)'
xLabel = 'Driving Signals'

dataFeature7kV = dataFeature.loc[(dataFeature['Voltage_kV'] == 7)]
meanRawPeakF, stdRawPeakF = computeErrorbar1Var(dataFeature7kV, 'DriveSignal', driveSignals, 'RawPeakForce')
meanSteadyRawF, stdSteadyRawF = computeErrorbar1Var(dataFeature7kV, 'DriveSignal', driveSignals, 'steadyRawForce')

print("\n ----- Mixed data  ----- ")
print("Raw force - Mean (n=10, 110mm):")
print(meanRawPeakF)
print("Raw force - STD (n=10, 110mm):")
print(stdRawPeakF)

print("Steady State Raw force - Mean (n=10, 110mm):")
print(meanSteadyRawF)
print("Steady State Raw force - STD (n=10, 110mm):")
print(stdSteadyRawF)

colors = sns.light_palette(pltBlue, reverse=True, n_colors=(xLen+8)) # Blue for peak values
colors2 = sns.light_palette(pltGreen, reverse=True, n_colors=(xLen+8)) # Green for steady state values

fig2, ax2 = plt.subplots(dpi=300, figsize=(2,2))    

for i in range(xLen):
    ax2.plot(x[i]+0.05, meanRawPeakF[i], '.', c=colors[xLen])
    ax2.errorbar(x[i]+0.05, meanRawPeakF[i], yerr=stdRawPeakF[i], fmt='none', ecolor=colors[xLen], capsize=0)
    
    ax2.plot(x[i]-0.05, meanSteadyRawF[i], '.', c=colors2[xLen])
    ax2.errorbar(x[i]-0.05, meanSteadyRawF[i], yerr=stdSteadyRawF[i], fmt='none', ecolor=colors2[xLen], capsize=0)

ax2.set_ylabel(yLabelStr)
ax2.set_xlabel(xLabel)
ax2.set_xticks(x)
ax2.set_xticklabels(xLabelStr);

ax2.set_xlim([x[0]-0.5, x[-1]+0.5])
ax2.set_ylim([0, 10])

fig2.savefig("drive_signal_comparison.pdf", bbox_inches='tight')

In [ ]:
'''
Analysis for Visualizing the Errorbar of multiple conditions (driving signals) - rising time
'''
driveSignals = ['pulse','triangle','ramp']

colorSet = 'hls'
style = '.-'

xLen = len(driveSignals)
x = np.arange(xLen)
xLabelStr = driveSignals
yLabelStr = 'Rise Time (s)'
xLabel = 'Driving Signals'

dataFeature7kV = dataFeature.loc[(dataFeature['Voltage_kV'] == 7)]
meanRawPeakF, stdRawPeakF = computeErrorbar1Var(dataFeature7kV, 'DriveSignal', driveSignals, 'TriseForce')

print("\n ----- Mixed data  ----- ")
print("Rising time - Mean (n=10, 110mm):")
print(meanRawPeakF)
print("Rising time - STD (n=10, 110mm):")
print(stdRawPeakF)


colors = sns.light_palette(pGrey, reverse=True, n_colors=(xLen+8)) # Blue for peak values

fig2, ax2 = plt.subplots(dpi=300, figsize=(2,2))    

for i in range(xLen):
    ax2.plot(x[i]+0.05, meanRawPeakF[i], '.', c=colors[xLen-i])
    ax2.errorbar(x[i]+0.05, meanRawPeakF[i], yerr=stdRawPeakF[i], fmt='none', ecolor=colors[xLen-i], capsize=0)

ax2.set_ylabel(yLabelStr)
ax2.set_xlabel(xLabel)
ax2.set_xticks(x)
ax2.set_xticklabels(xLabelStr);

ax2.set_xlim([x[0]-0.5, x[-1]+0.5])

fig2.savefig("drive_signal_comparison_riseTime.pdf", bbox_inches='tight')

In [ ]:
'''Plot signal waveforms (Ramping difference)'''

aSignal = nanoData.loc[(nanoData['Label'] == '110.0mm7.0kVpulse'), 'Data'].values[0][0]
aSignal2 = nanoData.loc[(nanoData['Label'] == '110.0mm7.0kVtriangle'), 'Data'].values[0][0]
aSignal3 = nanoData.loc[(nanoData['Label'] == '110.0mm7.0kVramp'), 'Data'].values[0][0]

fig1, ax1 = plt.subplots(dpi=300, figsize=(4,2))

ax1.plot(np.arange(aSignal.shape[0])/Fs-1, -aSignal[:,2]+aSignal[0,2], c=pltPurple)
ax1.plot(np.arange(aSignal2.shape[0])/Fs, -aSignal2[:,2]+aSignal2[0,2], c=pltYellow)
ax1.plot(np.arange(aSignal3.shape[0])/Fs, -aSignal3[:,2]+aSignal3[0,2], c=pltRed)

ax1.set_xlim([0, 4.0])
ax1.set_xlabel('Time (s)')
ax1.set_ylim([-0.5, 9.5])
ax1.set_ylabel('Force (N)')

ax1.legend(labels=['Pulse','Triangle','Ramp'], frameon=False, bbox_to_anchor=(1.0, 0.9), loc='upper left');

fig1.savefig("drive_signal_waveforms.pdf", bbox_inches='tight')

In [ ]:
'''Plot input waveforms (Ramping difference)'''
measureDataPath = './ForceData202405Ramp' # driving signal types (pulse, triangle, ramp)

for root, directories, files in walk(measureDataPath):
    for fileName in files:       
        driveSignal = decodeData(fileName, '\w+', rearCode='_', isString=True)
        if driveSignal is not None and len(driveSignal)>4:
            driveSignal = driveSignal[:-5]
            
        t_shift = 0
        if driveSignal == 'Ramp':
            t_shift = -0.076
        elif driveSignal == 'Square':
            t_shift = -0.08
        elif driveSignal == 'Triangle':
            t_shift = 0.045
        
        readData = pd.read_csv(ospa.join(root, fileName), header = None)
        voltageData = []
        for i in range(3):
            voltageData.append(readData[i].to_numpy(dtype='float'))
            
#         _,ax = plt.subplots(3,1,dpi=300, figsize=(16,6))
        _,ax = plt.subplots(3,1,dpi=72, figsize=(16,6))
        for i in range(3):
#             ax[i].plot(np.arange(voltageData[i].shape[0])/Fs + t_shift, voltageData[i])
            ax[i].plot(voltageData[i])
            ax[i].set_ylabel("%s %d" % (driveSignal, i))
#             ax[i].set_xlim([0.95, 2.05])
#             ax[i].set_xticks(np.arange(0.95,2.05,0.05))
#             ax[i].set_xticks(np.arange(2500,3500,100))
        ax[2].set_xlabel('Time (sec)')
            
